In [28]:
import requests
import time
from pprint import pprint


ip='3.136.18.23'

req_body = {
    "domain":"(define (domain blocksworld) (:requirements :strips) (:predicates (on ?x ?y) (ontable ?x) (clear ?x) (arm-free) (holding ?x) ) (:action pick-up :parameters (?x) :precondition (and (clear ?x) (ontable ?x) (arm-free)) :effect (and (not (ontable ?x)) (not (clear ?x)) (not (arm-free)) (holding ?x))) (:action put-down :parameters (?x) :precondition (holding ?x) :effect (and (not (holding ?x)) (clear ?x) (arm-free) (ontable ?x))) (:action stack :parameters (?x ?y) :precondition (and (holding ?x) (clear ?y)) :effect (and (not (holding ?x)) (not (clear ?y)) (clear ?x) (arm-free) (on ?x ?y))) (:action unstack :parameters (?x ?y) :precondition (and (on ?x ?y) (clear ?x) (arm-free)) :effect (and (holding ?x) (clear ?y) (not (clear ?x)) (not (arm-free)) (not (on ?x ?y)))))",
    "problem":"(define (problem blocksworld-4-2) (:domain blocksworld) (:objects b d c a ) (:init (clear a) (clear c) (clear d) (ontable a) (ontable b) (ontable d) (on c b) (arm-free)) (:goal (and (on a b) (on b c) (on c d))))"
}





In [29]:
def test_planner(package_name,req_body,service="solve"):
    service_url="http://"+ip+":5001/package/"+package_name+"/"+service
    solve_request = requests.post(service_url, json=req_body).json()
    celery_result = requests.get("http://"+ip+":5001" + solve_request['result'])
    print('Computing...')
    while celery_result.text == 'PENDING':
        celery_result = requests.get("http://"+ip+":5001" + solve_request['result'])
        time.sleep(0.5)
    pprint(celery_result.json())

In [30]:
# lama
test_planner("lama",req_body)

Computing...
{'result': {'call': 'lama domain problem',
            'output': {'sas_plan.1': '(unstack c b)\n'
                                     '(stack c d)\n'
                                     '(pick-up b)\n'
                                     '(stack b c)\n'
                                     '(pick-up a)\n'
                                     '(stack a b)\n'
                                     '; cost = 6 (unit cost)\n'},
            'stderr': '',
            'stdout': 'INFO     Running translator.\n'
                      'INFO     translator stdin: None\n'
                      'INFO     translator time limit: None\n'
                      'INFO     translator memory limit: None\n'
                      'INFO     translator command line string: '
                      '/usr/bin/python3 '
                      '/workspace/downward/builds/release/bin/translate/translate.py '
                      'domain problem --sas-file output.sas\n'
                      'Parsing...

In [31]:
# kstar
test_planner("kstar",req_body)

Computing...
{'result': {'call': "kstar domain problem --search 'kstar(blind(),k=1)'",
            'output': {'sas_plan.1': '(unstack c b)\n'
                                     '(stack c d)\n'
                                     '(pick-up b)\n'
                                     '(stack b c)\n'
                                     '(pick-up a)\n'
                                     '(stack a b)\n'
                                     '; cost = 6 (unit cost)\n',
                       'sas_plan.2': '(unstack c b)\n'
                                     '(stack c d)\n'
                                     '(pick-up b)\n'
                                     '(stack b c)\n'
                                     '(pick-up a)\n'
                                     '(stack a b)\n'
                                     '(unstack a b)\n'
                                     '(stack a b)\n'
                                     '; cost = 8 (unit cost)\n'},
            'stderr': '',
        

In [32]:
# lama-first
test_planner("lama-first",req_body)

Computing...
{'result': {'call': 'lama-first domain problem',
            'output': {'sas_plan': '(unstack c b)\n'
                                   '(stack c d)\n'
                                   '(pick-up b)\n'
                                   '(stack b c)\n'
                                   '(pick-up a)\n'
                                   '(stack a b)\n'
                                   '; cost = 6 (unit cost)\n'},
            'stderr': '',
            'stdout': 'INFO     Running translator.\n'
                      'INFO     translator stdin: None\n'
                      'INFO     translator time limit: None\n'
                      'INFO     translator memory limit: None\n'
                      'INFO     translator command line string: '
                      '/usr/bin/python3 '
                      '/workspace/downward/builds/release/bin/translate/translate.py '
                      'domain problem --sas-file output.sas\n'
                      'Parsing...\n'
    

In [33]:
# cerberus-sat
test_planner("cerberus-sat",req_body)

Computing...
{'result': {'call': 'cerberus-sat domain problem',
            'output': {'sas_plan.1': '(unstack c b)\n'
                                     '(stack c d)\n'
                                     '(pick-up b)\n'
                                     '(stack b c)\n'
                                     '(pick-up a)\n'
                                     '(stack a b)\n'
                                     '; cost = 6 (unit cost)\n'},
            'stderr': '',
            'stdout': "['translate', 'transform_task', 'search']\n"
                      'translate\n'
                      "['translate', 'transform_task', 'search']\n"
                      'INFO     Running translator.\n'
                      'INFO     translator stdin: None\n'
                      'INFO     translator time limit: None\n'
                      'INFO     translator memory limit: None\n'
                      'INFO     translator command line string: '
                      '/usr/bin/python3 '
   

In [34]:
# enhsp-2020
test_planner("enhsp-2020",req_body)

Computing...
{'result': {'call': 'enhsp-2020 -o domain -f problem >> sas_plan.1',
            'output': {'sas_plan.1': 'Domain parsed\n'
                                     'Problem parsed\n'
                                     'Grounding..\n'
                                     'Grounding Time: 61\n'
                                     '(Pre Simplification) - |A|+|P|+|E|: 40\n'
                                     'AIBR :: Number of Supporters = 40\n'
                                     '(After Easy Simplification) - '
                                     '|A|+|P|+|E|: 40\n'
                                     '|F|:29\n'
                                     '|X|:0\n'
                                     'Grounding and Simplification finished\n'
                                     '|A|:40\n'
                                     '|P|:0\n'
                                     '|E|:0\n'
                                     'ha:false htfalse\n'
                                     '

In [37]:
# forbiditerative-topk 1 plan
topk_body=req_body.copy()
topk_body["number_of_plans"]=1
test_planner("forbiditerative-topk",topk_body)

Computing...
{'result': {'call': 'forbiditerative-topk domain problem 3',
            'output': {'sas_plan.1': '(unstack c b)\n'
                                     '(stack c d)\n'
                                     '(pick-up b)\n'
                                     '(stack b c)\n'
                                     '(pick-up a)\n'
                                     '(stack a b)\n'
                                     '; cost = 6 (unit cost)\n',
                       'sas_plan.2': '(unstack c b)\n'
                                     '(stack c d)\n'
                                     '(pick-up b)\n'
                                     '(stack b c)\n'
                                     '(pick-up a)\n'
                                     '(stack a b)\n'
                                     '(unstack a b)\n'
                                     '(stack a b)\n'
                                     '; cost = 8 (unit cost)\n',
                       'sas_plan.3': '(unstack c

In [38]:
# forbiditerative-topk 3 plan
topk_body=req_body.copy()
topk_body["number_of_plans"]=3
test_planner("forbiditerative-topk",topk_body)

Computing...
{'result': {'call': 'forbiditerative-topk domain problem 3',
            'output': {'sas_plan.1': '(unstack c b)\n'
                                     '(stack c d)\n'
                                     '(pick-up b)\n'
                                     '(stack b c)\n'
                                     '(pick-up a)\n'
                                     '(stack a b)\n'
                                     '; cost = 6 (unit cost)\n',
                       'sas_plan.2': '(unstack c b)\n'
                                     '(stack c d)\n'
                                     '(pick-up b)\n'
                                     '(stack b c)\n'
                                     '(pick-up a)\n'
                                     '(stack a b)\n'
                                     '(unstack a b)\n'
                                     '(stack a b)\n'
                                     '; cost = 8 (unit cost)\n',
                       'sas_plan.3': '(unstack c